In [111]:
import pandas as pd
import re


In [112]:
json_file_path = "/Users/ezequielesparza/TKH-Social-Media-Engagement-Analytics/data/dataset_instagram-scraper_2023-04-11_20-38-43-027.json"
df = pd.read_json(json_file_path)
pd.set_option('display.max_columns', None)

In [113]:
# drop columns with over 190 null values
df = df.drop(['firstComment','latestComments','caption','mentions', 'shortCode', 'url', 'dimensionsHeight', 'dimensionsWidth', 'displayUrl', 'images', 'childPosts', 'ownerFullName', 'ownerUsername', 'ownerId', 'isSponsored', 'alt', 'videoUrl', 'locationName', 'locationId', 'productType','taggedUsers' ,'title', 'description'], axis=1)

In [117]:
# CLEANING HASHTAGS COLUMN

# Explode the 'hashtags' column in the original DataFrame
df_exploded = df.explode('hashtags')

# Clean the hashtags by converting to lowercase and removing non-alphabetic characters
df_exploded['cleaned_hashtags'] = df_exploded['hashtags'].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z]', '', x.lower()))

# Drop the original 'hashtags' column
df_exploded.drop('hashtags', axis=1, inplace=True)

# Rename the 'cleaned_hashtags' column back to 'hashtags'
df_exploded.rename(columns={'cleaned_hashtags': 'hashtags'}, inplace=True)

# Save the cleaned data to a CSV file
df_exploded.to_csv('IG_cleaned_data.csv', index=False)

# Confirm the CSV file is saved
print("CSV file 'IG_cleaned_data.csv' saved successfully.")

CSV file 'IG_cleaned_data.csv' saved successfully.


In [109]:
# WITHOUT HASHTAGS

# Drop the 'hashtags' column and remove duplicate rows
without_hashtags = df_exploded.drop('hashtags', axis=1).drop_duplicates()

# Save 'without_hashtags' to a CSV file
without_hashtags.to_csv('ig_without_hashtags.csv', index=False)

In [110]:
# JUST HASHTAGS

# Extract the 'hashtags' column from df_exploded
hashtags = df_exploded['hashtags']

# Get unique hashtags and assign them an index
just_hashtags = pd.DataFrame({'hashtags': hashtags.unique()})
just_hashtags['index'] = just_hashtags.reset_index().index

# Reorder the columns
just_hashtags = just_hashtags[['index', 'hashtags']]

# Save 'just_hashtags' to a CSV file
just_hashtags.to_csv('ig_just_hashtags.csv', index=False)

# Confirm the CSV file is saved
print("CSV file 'just_hashtags.csv' saved successfully.")



CSV file 'just_hashtags.csv' saved successfully.


In [102]:
# Merge 'df_exploded' with 'just_hashtags' to create the primary table
primary = pd.merge(df_exploded[['id', 'hashtags']], just_hashtags, left_on='hashtags', right_on='hashtags', how='left')

# Add an 'index' column numbering each row
primary['index'] = range(len(primary))

# Replace NaN values in the 'hashtags' column with a string representation
primary['hashtags'] = primary['hashtags'].fillna('nan')

# Create a mapping dictionary to store the index values for each hashtag
hashtag_index_mapping = just_hashtags.set_index('hashtags')['index'].to_dict()

# Map the correct index values to the 'hashtag_index' column in the primary table
primary['hashtag_index'] = primary['hashtags'].map(hashtag_index_mapping)

# Drop the 'hashtags' column from the primary table
primary.drop('hashtags', axis=1, inplace=True)

# Reorder the columns to have 'index', 'id', and 'hashtag_index'
primary = primary[['index', 'id', 'hashtag_index']]

# Save the primary table to a CSV file
primary.to_csv('IG_primary_table.csv', index=False)

# Confirm the CSV file is saved
print("CSV file 'IG_primary_table.csv' saved successfully.")


CSV file 'IG_primary_table.csv' saved successfully.


In [104]:
# CHECKING TO SEE IF DATA IS ACCURATE

# Load the CSV files
cleaned_data = pd.read_csv('/Users/ezequielesparza/TKH-Social-Media-Engagement-Analytics/extract/IG_cleaned_data.csv')
just_hashtags = pd.read_csv('/Users/ezequielesparza/TKH-Social-Media-Engagement-Analytics/extract/just_hashtags.csv')
primary_table = pd.read_csv('/Users/ezequielesparza/TKH-Social-Media-Engagement-Analytics/extract/IG_primary_table.csv')

# Merge primary table with just_hashtags table to get the expected mapping
expected_mapping = pd.merge(primary_table, just_hashtags, left_on='hashtag_index', right_on='index')

# Create an empty DataFrame to store mappings and correctness
mapping_results = pd.DataFrame(columns=['id', 'hashtag_index', 'correct', 'actual_hashtags'])

# Verify data mapping
for index, row in expected_mapping.iterrows():
    id_value = row['id']
    expected_hashtag_index = row['hashtag_index']
    expected_hashtag = row['hashtags']
    
    # Find the corresponding entry in the original data
    original_entry = cleaned_data[(cleaned_data['id'] == id_value) & (cleaned_data['hashtags'] == expected_hashtag)]
    
    if len(original_entry) > 0:
        mapping_results = mapping_results.append({'id': id_value, 'hashtag_index': expected_hashtag_index, 'correct': True, 'actual_hashtags': ''}, ignore_index=True)
    else:
        actual_hashtags = cleaned_data[cleaned_data['id'] == id_value]['hashtags'].astype(str).tolist()
        mapping_results = mapping_results.append({'id': id_value, 'hashtag_index': expected_hashtag_index, 'correct': False, 'actual_hashtags': ', '.join(actual_hashtags)}, ignore_index=True)

# Print the number of correct and incorrect mappings found
correct_count = len(mapping_results[mapping_results['correct'] == True])
incorrect_count = len(mapping_results[mapping_results['correct'] == False])
print(f"Total correct mappings: {correct_count}")
print(f"Total incorrect mappings: {incorrect_count}")


/var/folders/fy/fl5q9lyx5cl_4sqy67dvl7f80000gn/T/ipykernel_90321/1206941576.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mapping_results = mapping_results.append({'id': id_value, 'hashtag_index': expected_hashtag_index, 'correct': True, 'actual_hashtags': ''}, ignore_index=True)
/var/folders/fy/fl5q9lyx5cl_4sqy67dvl7f80000gn/T/ipykernel_90321/1206941576.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mapping_results = mapping_results.append({'id': id_value, 'hashtag_index': expected_hashtag_index, 'correct': True, 'actual_hashtags': ''}, ignore_index=True)
/var/folders/fy/fl5q9lyx5cl_4sqy67dvl7f80000gn/T/ipykernel_90321/1206941576.py:24: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype

Total correct mappings: 1656
Total incorrect mappings: 0
